In [ ]:
from bs4 import BeautifulSoup
import unicodedata
import requests
import json

def limpiar_unicode(texto):
    if texto:
        return unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('ascii')
    return texto

def obtener_precio_actual(soup):
    try:
        precio_actual = soup.find('span', {'class': 'a-price-whole'}).get_text(strip=True)
        precio_decimal = soup.find('span', {'class': 'a-price-fraction'}).get_text(strip=True)
        precio_actual = precio_actual + '.' + precio_decimal + ' €'
        return limpiar_unicode(precio_actual)
    except:
        return 'No disponible'

def obtener_precio_anterior(soup):
    try:
        precio_anterior = soup.find('span', {'class': 'a-price a-text-price'}).find('span', {'class': 'a-offscreen'}).get_text(strip=True)
        return limpiar_unicode(precio_anterior)
    except:
        return 'No disponible'

def obtener_detalles_tecnicos(soup):
    detalles_tecnicos = {}
    try:
        tabla_detalles = soup.find('table', {'id': 'productDetails_techSpec_section_1'})
        if tabla_detalles:
            filas = tabla_detalles.find_all('tr')
            for fila in filas:
                clave = fila.find('th').get_text(strip=True)
                valor = fila.find('td').get_text(strip=True)
                detalles_tecnicos[limpiar_unicode(clave)] = limpiar_unicode(valor)
        else:
            detalles_tecnicos = 'No disponible'
    except:
        detalles_tecnicos = 'No disponible'
    return detalles_tecnicos

def obtener_soup(api_key, product_id):
    url = 'https://www.amazon.es/dp/' + product_id
    params = {
        'api_key': api_key,
        'url': url,
        'country_code': 'ES'
    }
    response = requests.get('https://api.scraperapi.com/', params=params)
    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    else:
        print(f'Error: {response.status_code}')
        return None

def obtener_datos_producto(api_key, product_id):
    soup = obtener_soup(api_key, product_id)
    if soup:
        precio_actual = obtener_precio_actual(soup)
        precio_anterior = obtener_precio_anterior(soup)
        detalles_tecnicos = obtener_detalles_tecnicos(soup)
        resultado = {
            'product_id': product_id,
            'precio_actual': precio_actual,
            'precio_anterior': precio_anterior,
            'detalles_tecnicos': detalles_tecnicos
        }
        return resultado
    return None

# Cargar los IDs desde el archivo JSON
with open("/home/aluvesprada/Escritorio/VISUAL_STUDIO/PIA/PROJECTE/PIA_Projectes-main/Projecte_01/scrapping/ids.json", 'r') as file:
    product_ids = json.load(file)

# API_KEY
API_KEY = '83bc7b5f85e1d83b695e0d39816ebb77'

# Guardar los datos conforme se obtienen
with open("mobiles.json", "w", encoding='utf-8') as outfile:
    outfile.write("[")
    first = True
    for product_id in product_ids:
        detalles = obtener_datos_producto(API_KEY, product_id)
        if detalles:
            if not first:
                outfile.write(",\n")
            json.dump(detalles, outfile, ensure_ascii=False, indent=4)
            first = False
    outfile.write("\n]")

print("Datos guardados en mobiles.json")
